### RGCN 
In this notebook we want to make an easy implementation of the RGCN model from scratch - namely using only Pytorch (no pyg or dgl or whatever)

The first task will probably be defining a class to define a RGCN layer. 

In order to do that we will have to remember what we need!

- 

In [2]:
#import libraries
import torch 
import torch.nn as nn
import pandas as pd
import numpy as np
from tqdm import tqdm
